In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import datetime

In [2]:
with open("request.txt", "r", encoding="utf-8", errors="ignore") as file:
    contents = file.read()

In [3]:
soup = BeautifulSoup(contents, 'html.parser')

In [4]:
# Encontre todos os blocos de informações do jogo
jogo_blocos = soup.find_all('tr', class_='app')

In [5]:
data = []
for jogo in jogo_blocos:
    appid = jogo.get('data-appid')
    nome = jogo.find('a', class_='b').text.strip()
    desconto = jogo.find_all('td')[3].text.strip().replace('%', '').replace('-', '')
    desconto = float(desconto) if desconto else None
    preco = jogo.find_all('td')[4].text.strip().replace('R$ ', '').replace(',', '.')
    preco = float(preco) if preco else None
    rating = jogo.find_all('td')[5].text.strip().replace('%', '')
    rating = float(rating) if rating else None
    end = jogo.find_all('td')[6].text.strip()
    started = jogo.find_all('td')[7].text.strip()
    release = jogo.find_all('td')[8].text.strip()

    # Nova linha para extrair a informação 'new historical low'
    new_historical_low = jogo.find('span', class_='highest-discount-major')
    new_historical_low = new_historical_low.text.strip() if new_historical_low else 'N/A'


    # Crie um dicionário com as informações e adicione à lista de dados
    data.append({
        'ID': appid,
        'Nome': nome,
        'Desconto': desconto,
        'Preco': preco,
        'Rating': rating,
        'End': end,
        'Started': started,
        'Release': release,
        'New_Historical_Low': new_historical_low 
    })

In [6]:
# Crie um DataFrame a partir da lista de dados
df = pd.DataFrame(data)

In [7]:
df.head()

,ID,Nome,Desconto,Preco,Rating,End,Started,Release,New_Historical_Low
0,1092790,Inscryption,64.0,28.97,95.54,in 7 days,7 days ago,Oct 2021,new historical low
1,1061910,Metal: Hellsinger,50.0,49.99,94.21,in 7 days,7 days ago,Sep 2022,new historical low
2,1451940,NEEDY STREAMER OVERLOAD,50.0,15.49,93.17,in 7 days,17 days ago,Jan 2022,new historical low
3,1971650,OCTOPATH TRAVELER II,53.0,202.91,93.03,in 7 days,7 days ago,Feb 2023,new historical low
4,1369630,ENDER LILIES: Quietus of the Knights,50.0,24.74,92.53,in 7 days,17 days ago,Jun 2021,new historical low


In [8]:
df.isnull().sum() / df.shape[0]

ID                    0.0
Nome                  0.0
Desconto              0.0
Preco                 0.0
Rating                0.0
End                   0.0
Started               0.0
Release               0.0
New_Historical_Low    0.0
dtype: float64

In [9]:
df.New_Historical_Low.value_counts()

N/A                   4041
new historical low     291
Name: New_Historical_Low, dtype: int64

In [10]:
porcentagem_na = df.apply(lambda x: (x == 'N/A').mean() * 100)

In [11]:
porcentagem_na

ID                     0.000000
Nome                   0.000000
Desconto               0.000000
Preco                  0.000000
Rating                 0.000000
End                    0.000000
Started                0.000000
Release                0.000000
New_Historical_Low    93.282548
dtype: float64